In [3]:
import rasterio as rio
import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist
import os

In [34]:
input_path = r"C:\Users\giova\OneDrive\Desktop\ZEULI\5 Zeuli 220802\Raster Alberi"
output_path = r"C:\Users\giova\OneDrive\Desktop\ZEULI\5 Zeuli 220802\5 Zeuli 220802 ground truth.xlsx"
df_alberi = pd.read_excel(r"C:\Users\giova\OneDrive\Desktop\ZEULI\ZEULI_OLIVO_UTM_220802.xlsx")

In [25]:
def raster_to_df(tif_path, df_alberi):
    src = rio.open(tif_path) #apre l'immagine del singolo albero
    
    ndvi = src.read(12) #legge la banda relativa all'NDVI
    threshold = 0.2
    olivo_mask = ndvi>threshold #prende solo i pixels appartenenti all'ulivo

    data = []
    df_pixels = []
    
    for i in range(src.height):
        for j in range(src.width):
            if olivo_mask[i,j]: #cicla per ogni pixel appartenente all'ulivo
                UTM_lon, UTM_lat = src.xy(i, j)
                pixel_values = [src.read(b + 1, window=((i, i+1), (j, j+1)))[0][0] for b in range(src.count)] #lege i valori dei pixel nelle bande
                data.append({
                    'ID_Pixel': f'{i}_{j}',
                    'ID_Albero': '',
                    'UTM_lon': UTM_lon,
                    'UTM_lat': UTM_lat,
                    'Blue-444': pixel_values[0],
                    'Blue': pixel_values[1],
                    'Green-531': pixel_values[2],
                    'Green': pixel_values[3],
                    'NIR': pixel_values[4],
                    'Red-650': pixel_values[5],
                    'Rededge-705': pixel_values[6],
                    'Rededge-740': pixel_values[7],         #qui devo cambiare i nomi ed il numero delle bande
                    'Rededge': pixel_values[8],
                    'Red': pixel_values[9],
                    'Temp': pixel_values[10],
                    'NDVI': pixel_values[11],
                    'TEMP ARIA': '',
                    'UMID ARIA': '',
                    'SWP': ''
                })
    df_pixels = pd.DataFrame(data)

    lon_mean = df_pixels['UTM_lon'].mean()
    lat_mean = df_pixels['UTM_lat'].mean()
    
    # Calcola le distanze euclidee tra le medie e le coordinate UTM_lon e UTM_lat di ogni albero
    distances = cdist(np.array(df_alberi[['UTM_lon', 'UTM_lat']]), np.array([[lon_mean, lat_mean]]))
    
    # Trova l'indice dell'albero più vicino alle medie
    closest_tree_index = np.argmin(distances)
    
    # Prendi i valori ID e SWP dell'albero più vicino
    closest_tree_id = df_alberi.iloc[closest_tree_index]['ID']
    closest_tree_swp = df_alberi.iloc[closest_tree_index]['SWP']
    closest_tree_air_temp = df_alberi.iloc[closest_tree_index]['TEMP ARIA']
    closest_tree_air_hum = df_alberi.iloc[closest_tree_index]['UMID ARIA']
    
    #######qui devo aggiungere i valori di temperatura e umidità relativa
    
    # Assegna i valori di ID e SWP dell'albero più vicino a tutti i pixel nel DataFrame dei pixel
    df_pixels['ID_Albero'] = closest_tree_id
    df_pixels['TEMP ARIA'] = closest_tree_air_temp
    df_pixels['UMID ARIA'] = closest_tree_air_hum
    df_pixels['SWP'] = closest_tree_swp
    

    return df_pixels

In [35]:
df_final = pd.DataFrame()
tif_alberi = [img for img in os.listdir(input_path) if img.endswith('.tif')]
for tif_albero in tif_alberi:
    tif_path = os.path.join(input_path, tif_albero)
    df_temp = raster_to_df(tif_path, df_alberi)
    df_final = pd.concat([df_final, df_temp], ignore_index=True)

In [36]:
df_final

,ID_Pixel,ID_Albero,UTM_lon,UTM_lat,Blue-444,Blue,Green-531,Green,NIR,Red-650,Rededge-705,Rededge-740,Rededge,Red,Temp,NDVI,TEMP ARIA,UMID ARIA,SWP
0,6_31,136,600452.8721,4.566153e+06,33596.0,24482.0,26412.0,26782.0,31573.0,32004.0,34402.0,34002.0,36861.0,20599.0,43.366352,0.210343,34.2,37.1,30.2
1,6_32,136,600452.9521,4.566153e+06,35451.0,27335.0,22145.0,27190.0,31276.0,29988.0,31856.0,33036.0,37974.0,20151.0,43.394917,0.216326,34.2,37.1,30.2
2,7_32,136,600452.9521,4.566153e+06,30772.0,21322.0,21499.0,24915.0,30328.0,26632.0,31488.0,31539.0,34675.0,16798.0,43.896461,0.287103,34.2,37.1,30.2
3,7_33,136,600453.0321,4.566153e+06,31930.0,23820.0,19103.0,25375.0,27365.0,25788.0,26270.0,30255.0,35908.0,15201.0,43.919594,0.285768,34.2,37.1,30.2
4,8_32,136,600452.9521,4.566152e+06,27598.0,16890.0,24658.0,25591.0,27544.0,22293.0,29764.0,29302.0,30404.0,17286.0,43.881767,0.228820,34.2,37.1,30.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134572,66_33,60,599810.8721,4.565661e+06,33148.0,27853.0,28475.0,32842.0,42214.0,33826.0,26752.0,41711.0,46449.0,17421.0,51.383881,0.415746,33.5,37.7,13.0
134573,66_34,60,599810.9521,4.565661e+06,32787.0,22670.0,28103.0,34071.0,40815.0,30858.0,25174.0,41831.0,46741.0,19556.0,52.898170,0.352139,33.5,37.7,13.0
134574,67_32,60,599810.7921,4.565661e+06,33878.0,31633.0,33180.0,32877.0,44458.0,34083.0,32586.0,36914.0,42564.0,25274.0,51.779484,0.275110,33.5,37.7,13.0
134575,67_33,60,599810.8721,4.565661e+06,35361.0,29500.0,30419.0,33282.0,42663.0,34321.0,29290.0,38308.0,42314.0,19689.0,52.955444,0.368457,33.5,37.7,13.0


In [37]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134577 entries, 0 to 134576
Data columns (total 19 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   ID_Pixel     134577 non-null  object 
 1   ID_Albero    134577 non-null  int64  
 2   UTM_lon      134577 non-null  float64
 3   UTM_lat      134577 non-null  float64
 4   Blue-444     134577 non-null  float32
 5   Blue         134577 non-null  float32
 6   Green-531    134577 non-null  float32
 7   Green        134577 non-null  float32
 8   NIR          134577 non-null  float32
 9   Red-650      134577 non-null  float32
 10  Rededge-705  134577 non-null  float32
 11  Rededge-740  134577 non-null  float32
 12  Rededge      134577 non-null  float32
 13  Red          134577 non-null  float32
 14  Temp         134577 non-null  float32
 15  NDVI         134577 non-null  float32
 16  TEMP ARIA    134577 non-null  float64
 17  UMID ARIA    134577 non-null  float64
 18  SWP          134577 non-

In [38]:
df_final.isnull().sum()

ID_Pixel       0
ID_Albero      0
UTM_lon        0
UTM_lat        0
Blue-444       0
Blue           0
Green-531      0
Green          0
NIR            0
Red-650        0
Rededge-705    0
Rededge-740    0
Rededge        0
Red            0
Temp           0
NDVI           0
TEMP ARIA      0
UMID ARIA      0
SWP            0
dtype: int64

In [39]:
df_final.nunique()

ID_Pixel        12617
ID_Albero          26
UTM_lon          1684
UTM_lat          2168
Blue-444        26353
Blue            23512
Green-531       28163
Green           29790
NIR             40323
Red-650         25243
Rededge-705     29730
Rededge-740     41378
Rededge         36826
Red             22981
Temp           132120
NDVI           133720
TEMP ARIA          17
UMID ARIA          24
SWP                26
dtype: int64

In [40]:
df_final.describe()

,ID_Albero,UTM_lon,UTM_lat,Blue-444,Blue,Green-531,Green,NIR,Red-650,Rededge-705,Rededge-740,Rededge,Red,Temp,NDVI,TEMP ARIA,UMID ARIA,SWP
count,134577.000000,134577.000000,1.345770e+05,134577.000000,134577.000000,134577.000000,134577.000000,134577.000000,134577.000000,134577.000000,134577.000000,134577.000000,134577.000000,134577.000000,134577.000000,134577.000000,134577.000000,134577.000000
mean,72.448784,599998.453434,4.565903e+06,11144.236328,9826.898438,12864.701172,12902.881836,31215.062500,9012.419922,13659.298828,26076.898438,20878.726562,8168.852539,39.167458,0.591601,33.986861,37.303603,25.022806
std,45.627655,254.786345,2.667912e+02,6363.897949,5448.839355,6799.550781,7348.586914,10983.717773,6247.885254,7307.427246,11529.204102,9277.004883,5438.255859,6.427470,0.175550,0.756437,1.503351,8.893590
min,32.000000,599748.952100,4.565492e+06,0.000000,0.000000,42.000000,0.000000,6121.000000,0.000000,806.000000,2855.000000,2530.000000,0.000000,30.672695,0.200014,32.700000,34.600000,10.200000
25%,37.000000,599802.552100,4.565643e+06,6796.000000,6019.000000,7977.000000,7696.000000,23491.000000,4914.000000,8504.000000,16897.000000,14037.000000,4466.000000,34.323280,0.469142,33.500000,36.100000,17.200000
50%,51.000000,599947.352100,4.566060e+06,9278.000000,8311.000000,11323.000000,11095.000000,33382.000000,7130.000000,11999.000000,27854.000000,20476.000000,6439.000000,36.794186,0.623943,33.700000,37.400000,24.600000
75%,137.000000,599965.752100,4.566130e+06,13681.000000,12251.000000,16395.000000,16533.000000,39755.000000,10998.000000,17402.000000,34933.000000,27118.000000,10130.000000,42.066750,0.731764,34.600000,38.200000,33.800000
max,148.000000,600465.192100,4.566179e+06,61343.000000,40689.000000,47274.000000,51253.000000,58705.000000,43534.000000,46431.000000,58069.000000,53371.000000,32275.000000,61.241749,1.000000,35.400000,40.600000,38.200000


In [41]:
df_ndvi = df_final[df_final['NDVI'] < 0.2]
df_ndvi.describe()

,ID_Albero,UTM_lon,UTM_lat,Blue-444,Blue,Green-531,Green,NIR,Red-650,Rededge-705,Rededge-740,Rededge,Red,Temp,NDVI,TEMP ARIA,UMID ARIA,SWP
count,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
df_final.to_excel(output_path, index=False)